In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
from torch import nn
import torch.nn.functional as F

In [6]:
df=pd.read_csv('Data/DataWithoutPadding.csv')

In [7]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [8]:
df.user_id.nunique()

2956

In [9]:
df.order_id.nunique()

189395

In [10]:
df.drop(columns=['user_id','order_number','order_id'],inplace=True)

In [11]:
df.drop(columns=['order_dow','order_hour_of_day','days_since_prior_order'],inplace=True)

In [12]:
df = df.fillna(0)

In [13]:
df

,45,196,260,329,432,651,890,1025,1158,1194,1244,1463,1511,1940,1999,2078,2086,2228,2295,2314,2452,2825,2855,2966,3298,3376,3464,3599,3896,3952,3957,4086,4210,4421,4472,4562,4605,4656,4799,4920,...,45123,45200,45535,45603,45633,46584,46616,46654,46667,46676,46720,46802,46820,46906,46969,46979,47042,47141,47144,47209,47626,47630,47672,47734,47759,47766,47977,48205,48364,48628,48679,48745,48775,49075,49175,49191,49235,49383,49520,49683
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
189391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
189392,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
189393,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Model

### creating sequence

In [14]:
seq=20
batch=df.shape[0]//seq
X=[]
y=[]
n=0
m=n+1
for i in range(batch):
    X.append(df[n:(n+seq)])
    if((m+seq)<len(df)):
        y.append(df[m:(m+seq)])
    else:
        y.append(df[n:(n+seq)])
    n=n+20
    m=n+1
#y[-1] = y[-1].append(df.iloc[0])

In [15]:
X = np.array([np.array(i) for i in X])
y = np.array([np.array(i) for i in y])

In [16]:
users = X.shape[0]//5
features = X.shape[-1]

In [17]:
#X_app = np.zeros([1,20,500])
#X = np.vstack((X,X_app))

In [18]:
X = X[:-4,:,:]
y = y[:-4,:,:]

In [19]:
X = X.reshape(users,5,20,features)
y = y.reshape(users,5,20,features)

In [20]:
test_idx = int(len(X)*(1-0.2))
test_idx

1514

In [21]:
X_train, X_test = X[:test_idx], X[test_idx:]
y_train, y_test = y[:test_idx], y[test_idx:]

In [22]:
X_test.shape

(379, 5, 20, 500)

In [23]:
val_idx = int(len(X_train)*(1-0.25))
val_idx

1135

In [24]:
X_data, X_val = X_train[:val_idx], X_train[val_idx:]
y_data, y_val = y_train[:val_idx], y_train[val_idx:]

In [25]:
print(X_train.shape)
print(y_train.shape)

(1514, 5, 20, 500)
(1514, 5, 20, 500)


In [26]:
print(X_val.shape)
print(y_val.shape)

(379, 5, 20, 500)
(379, 5, 20, 500)


In [27]:
print(X_test.shape)
print(y_test.shape)

(379, 5, 20, 500)
(379, 5, 20, 500)


In [ ]:
np.save('X_test_without_pad', X_test)
np.save('y_test_without_pad', y_test)

In [28]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU')

Training on GPU!


In [29]:
class InstaCartLSTM(nn.Module):
    
    def __init__(self, Data, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.orders = Data
        self.features = Data.shape[-1]
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(self.features, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, self.features)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm        
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [30]:
data = X_train.reshape(-1,X_train.shape[-1])
class_weight=[]
for i in range(data.shape[1]):
    positive=len(np.array(data[:,i][data[:,i]==1]))
    negative=len(np.array(data[:,i][data[:,i]==0]))
    class_weight.append(negative/positive)
class_weight = torch.as_tensor(class_weight, dtype=torch.float)

In [43]:
def train(net, X_train,y_train,X_val,y_val, epochs=10, batch_size=5, seq_length=100, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weight.cuda())
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.orders)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        train_set=len(X_train)
        
        for i in range(train_set):
            counter += 1
                        
            inputs, targets = torch.from_numpy(X_train[i]), torch.from_numpy(y_train[i])            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])            
            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model            
            output, h = net(inputs.float(), h)                                
            loss = criterion(output, targets.reshape(batch_size*seq_length,-1).float())
            #print(loss)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                
                validation_set = len(X_val)
                for i in range(validation_set):                    
                    x, y = torch.from_numpy(X_val[i]), torch.from_numpy(y_val[i])
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs.float(), val_h)
                    val_loss = criterion(output,targets.reshape(batch_size*seq_length,-1).float())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [47]:
# define and print the net
n_hidden=1024
n_layers=2

net = InstaCartLSTM(X_train, n_hidden, n_layers)
net = net.float()
print(net)

InstaCartLSTM(
  (lstm): LSTM(500, 1024, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1024, out_features=500, bias=True)
)


In [48]:
batch_size = 5
seq_length = 20
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, X_train,y_train,X_val,y_val,epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=1000)

Epoch: 1/50... Step: 1000... Loss: 1.3034... Val Loss: 1.2586
Epoch: 2/50... Step: 2000... Loss: 1.1472... Val Loss: 1.1718
Epoch: 2/50... Step: 3000... Loss: 1.1602... Val Loss: 1.0824
Epoch: 3/50... Step: 4000... Loss: 1.0715... Val Loss: 1.0969
Epoch: 4/50... Step: 5000... Loss: 1.0868... Val Loss: 1.0244
Epoch: 4/50... Step: 6000... Loss: 0.9045... Val Loss: 0.9370
Epoch: 5/50... Step: 7000... Loss: 1.0500... Val Loss: 0.9774
Epoch: 6/50... Step: 8000... Loss: 1.0785... Val Loss: 0.9114
Epoch: 6/50... Step: 9000... Loss: 1.0039... Val Loss: 0.8270
Epoch: 7/50... Step: 10000... Loss: 1.0508... Val Loss: 0.8922
Epoch: 8/50... Step: 11000... Loss: 0.7852... Val Loss: 0.8182
Epoch: 8/50... Step: 12000... Loss: 0.7219... Val Loss: 0.7498
Epoch: 9/50... Step: 13000... Loss: 0.7818... Val Loss: 0.8086
Epoch: 10/50... Step: 14000... Loss: 0.7241... Val Loss: 0.7528
Epoch: 10/50... Step: 15000... Loss: 0.8206... Val Loss: 0.6864
Epoch: 11/50... Step: 16000... Loss: 0.9217... Val Loss: 0.749

In [49]:
torch.save(net, 'Model/lstm_model_without_padding_1024.net')